<a href="https://colab.research.google.com/github/mitzenjeremywoo/google-colab-notebooks/blob/main/gemma3_RAG_implementation_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# --- Step 1: Install dependencies ---
!apt-get update
!apt-get install -y curl wget gnupg
!curl -fsSL https://ollama.com/install.sh | sh


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
!pip install langchain langchain_community faiss-cpu sentence-transformers

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [5]:
!nohup ollama serve &


nohup: appending output to 'nohup.out'


In [14]:
!ollama pull gemma3


In [11]:
# --- Step 3: Setup LangChain with FAISS ---
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.llms import Ollama

# Example docs (replace with your knowledge base later)
docs = [
    Document(page_content="VIP customers are entitled to a 60-day refund window."),
    Document(page_content="Standard customers get only a 30-day refund."),
    Document(page_content="Refunds are processed back to the original payment method."),
]

# Create embeddings + FAISS index
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)


/tmp/ipython-input-3125352728.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
# --- Step 4: Retrieval + Augmentation ---
query = "What is the refund policy for VIP customers?"
retrieved_docs = vectorstore.similarity_search(query, k=2)

context = "\n".join([doc.page_content for doc in retrieved_docs])
prompt = f"""
You are gemma3. Answer truthfully using the context below.

Context:
{context}

Question: {query}

Answer:
"""

print("==== Prompt Sent to Gemma 3 ====")
print(prompt)


==== Prompt Sent to Gemma 3 ====

You are gemma3. Answer truthfully using the context below.

Context:
VIP customers are entitled to a 60-day refund window.
Standard customers get only a 30-day refund.

Question: What is the refund policy for VIP customers?

Answer:



In [17]:
# --- Step 5: Call Gemma3 via Ollama ---
llm = Ollama(model="gemma3")
response = llm(prompt)
print("==== gemma3 Response ====")
print(response)


==== gemma3 3 Response ====
VIP customers are entitled to a 60-day refund window.
